In [2]:
import numpy as np
from sklearn import datasets
from sklearn.semi_supervised import LabelSpreading, SelfTrainingClassifier
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# iris = datasets.load_iris()
# rng = np.random.RandomState(42)
# random_unlabeled_points = rng.rand(len(iris.target)) < 0.3
# labels = np.copy(iris.target)
# labels[random_unlabeled_points] = -1
# label_prop_model.fit(iris.data, labels)

In [3]:
import os
datapath =  r'D:\projects\congress\processed\training_vectors\debatepol/'


In [4]:

ds_all = np.load(os.path.join(datapath, 'sentence_vectors_guns_nomoder.npy'))
parties_all = np.load(os.path.join(datapath, 'stance_lib_binary_guns_nomoder.npy'))
stance_all = np.load(os.path.join(datapath, 'stance_lib_7scale_guns_nomoder.npy'))
thetas_hard_all = np.load(os.path.join(datapath, 'topics_hard_guns_nomoder.npy'))
As_all = np.load(os.path.join(datapath, 'author_dummies_guns_nomoder.npy')) 
theta_categ= np.argmax(thetas_hard_all, axis = -1)
ds_all = np.float32(ds_all)
thetas_all = thetas_hard_all

In [5]:
ds_all.shape

(52675, 300)

In [6]:
def sample(parties_all, theta_categ, stance_all,biased = True, notmask = 0.8, extreme = 5):

    totalsamp = len(parties_all)

    if not biased: 
        sss = StratifiedShuffleSplit(n_splits=5, test_size= 1- notmask)
        train_id, test_id = next(sss.split(theta_categ, parties_all))
    else:
        p = np.array([(4- i)**extreme  if i <4 else (i -4)**extreme for i in stance_all])
        p = p / np.sum(p)
        train_id = np.random.choice(np.arange(totalsamp), int(totalsamp * notmask), replace = False, p = p)
        
#         orderid = np.argsort(stance_all)
#         tbn = int(len(parties_all) * notmask/2)
#         train_id = np.concatenate([orderid[:tbn], orderid[-tbn:]])
        
        test_id = [i for i in range(totalsamp) if i not in train_id]

    return train_id, test_id

In [7]:
trainid, testid = sample(parties_all, theta_categ, stance_all,biased = True, notmask =0.2)

In [8]:
len(trainid) + len(testid)

52675

In [9]:
# rounds = 6
# notmasks = [0.01, 0.03, 0.05, 0.08, 0.2, 0.4, 0.6, 0.8]

# # notmasks = [0.8]
# topn = 500000
# model_name = 'label_spreading'
# label_prop_model = LabelSpreading(kernel= 'knn', n_neighbors=10)
# for biased in [True, False]:
#   bia = 'bias'
#   if not biased:
#     bia = 'unbias'
#   f = open( r'./results/SKSSL/{}_{}.txt'.format(model_name, bia), 'w')
#   for notmask in notmasks:
#     version = "{}_{}_{}".format(model_name, notmask, bia)
#     print(version)
#     if biased:
# #       continue
#       trainid, testid = sample(labels, topics, biased = biased, notmask = notmask)

#       ys = np.copy(labels)
#       ys[testid] = -1

#       Xs = np.copy(sentvec_flt)
#       Xidx = np.arange(len(Xs))
      
#       np.random.shuffle(Xidx)
#       Xs = Xs[Xidx]
#       ys = ys[Xidx]
#       print(ys[:100])
#       label_prop_model.fit(Xs[:topn], ys[:topn])
#       Xt = sentvec_flt[testid]
#       yt = labels[testid]
#       y_pred = label_prop_model.predict(Xt)
#       acc = np.mean(y_pred == yt)
#       print(acc)
#       f.write(version)
#       f.write('\t'+str(acc))
#       f.write('\n')
#     else:
#       accs = []
#       for __ in range(rounds):
#         trainid, testid = sample(labels, topics, biased = biased, notmask = notmask)
#         ys = np.copy(labels)
#         ys[testid] = -1
        
#         Xs = np.copy(sentvec_flt)
#         Xidx = np.arange(len(Xs))
      
#         np.random.shuffle(Xidx)
#         Xs = Xs[Xidx]
#         ys = ys[Xidx]
#         print(ys[:100])
#         label_prop_model.fit(Xs[:topn], ys[:topn])
#         Xt = sentvec_flt[testid]
#         yt = labels[testid]
#         y_pred = label_prop_model.predict(Xt)
#         acc = np.mean(y_pred == yt)
#         print(acc)
#         accs.append(acc)
#       f.write(version)
#       for ac in accs:
#         f.write('\t'+str(ac))
#       f.write('\n')
#   f.close()

In [10]:
def predict_party_membership(s_pred, y, As):
    # filter out nan
    
    fin = np.isfinite(s_pred)
    s_pred = s_pred[fin]
    y = y[fin]
    As = As[fin]
    nonzero_inx = (np.sum(As.T, axis = -1) !=0)
    ave_slants = (np.matmul(As.T, s_pred)/(np.sum(As.T, axis = -1) + 0.000001))[nonzero_inx]
    y_ = np.matmul(As.T, y) >0 # this is party label per person, but those with 0 speech be 0
    print("test slant", ave_slants.mean(), ave_slants[:5])
    y_true = y_[nonzero_inx]
    y_pred = ave_slants>0
    return np.mean(y_true == y_pred)

In [11]:
# self training
extreme = 5

In [12]:
from sklearn.ensemble import GradientBoostingClassifier as GBC

In [13]:
rounds = 7
notmasks = [0.01, 0.03, 0.05, 0.08, 0.1, 0.2, 0.4, 0.6, 0.8]

# notmasks = [0.8]
topn = 5000000
model_name = 'self_training'


# notmasks = [0.01]
# topn = 5000
# rounds = 2

clf = RandomForestClassifier(max_depth=5, random_state=0)
self_training_model = SelfTrainingClassifier(clf)
# for biased in [True, False]:
for biased in [True, False]:
  bia = 'bias'
  if not biased:
    bia = 'unbias'
  f = open( r'D:\projects\congress\results\debatepol\train\SSL/{}_{}.txt'.format(model_name, bia), 'w')
  f.write('\taccs\tpccs\n')
  for notmask in notmasks:
    version = "{}_{}_{}".format(model_name, notmask, bia)
    print(version)
    
    if biased:
      accs = []
      pccs = []
      for __ in range(rounds):
        trainid, testid = sample(parties_all, theta_categ, stance_all, biased = biased, notmask = notmask, extreme = extreme)
        ys = np.copy(parties_all)
        ys[testid] = -1

        Xs = np.copy(ds_all)
        Xidx = np.arange(len(Xs))
      
        np.random.shuffle(Xidx)
        Xs = Xs[Xidx]
        ys = ys[Xidx]
        print(ys[:100])
        self_training_model.fit(Xs[:topn], ys[:topn])
        Xt = ds_all[testid]
        yt = parties_all[testid]
        At = As_all[testid]
        y_pred = self_training_model.predict(Xt)
        proba_pred = self_training_model.predict_proba(Xt)
        s_pred = np.array([np.log(i[1]*10001)- np.log(i[0]*1000) for i in proba_pred])

        acc = np.mean(y_pred == yt)
        accs.append(acc)
        
        pcc = predict_party_membership(s_pred, yt, At)
        pccs.append(pcc)
        
        
      f.write(version)

      f.write('\t' + str(accs))
      f.write('\t' + str(pccs))
      f.write('\n')
    else:
      accs = []
      pccs = []
      for __ in range(rounds):
        trainid, testid = sample(parties_all, theta_categ, stance_all, biased = biased, notmask = notmask, extreme = extreme)
        ys = np.copy(parties_all)
        ys[testid] = -1

        Xs = np.copy(ds_all)
        Xidx = np.arange(len(Xs))
      
        np.random.shuffle(Xidx)
        Xs = Xs[Xidx]
        ys = ys[Xidx]
        print(ys[:100])
        self_training_model.fit(Xs[:topn], ys[:topn])
        Xt = ds_all[testid]
        yt = parties_all[testid]
        At = As_all[testid]
        y_pred = self_training_model.predict(Xt)
        proba_pred = self_training_model.predict_proba(Xt)
        s_pred = np.array([np.log(i[1]*1000)- np.log(i[0]*1000) for i in proba_pred])

        acc = np.mean(y_pred == yt)
        accs.append(acc)
        
        pcc = predict_party_membership(s_pred, yt, At)
        pccs.append(pcc)
        
        
      f.write(version)

      f.write('\t' + str(accs))
      f.write('\t' + str(pccs))
      f.write('\n')
  f.close()

self_training_0.01_bias
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1]


KeyboardInterrupt: 

In [15]:
rounds = 5
notmasks = [0.01, 0.03, 0.05, 0.08, 0.1, 0.2, 0.4, 0.6, 0.8]

# notmasks = [0.8]
topn = 5000000


model_name = 'self_training_boosting'


# notmasks = [0.01]
# topn = 5000
# rounds = 2

# svc = SVC(probability=True, gamma="auto")
clf = GBC(n_estimators=75, learning_rate=1.0, max_depth=1, random_state=0)
self_training_model = SelfTrainingClassifier(clf)
# for biased in [True, False]:
for biased in [True, False]:
  bia = 'bias'
  if not biased:
    bia = 'unbias'
  f = open( r'D:\projects\congress\results\debatepol\train\SSL/{}_{}.txt'.format(model_name, bia), 'w')
  f.write('\taccs\tpccs\n')
  for notmask in notmasks:
    version = "{}_{}_{}".format(model_name, notmask, bia)
    print(version)
    
    if biased:
      accs = []
      pccs = []
      for __ in range(rounds):
        trainid, testid = sample(parties_all, theta_categ, stance_all, biased = biased, notmask = notmask, extreme = extreme)
        ys = np.copy(parties_all)
        ys[testid] = -1

        Xs = np.copy(ds_all)
        Xidx = np.arange(len(Xs))
      
        np.random.shuffle(Xidx)
        Xs = Xs[Xidx]
        ys = ys[Xidx]
        print(ys[:100])
        self_training_model.fit(Xs[:topn], ys[:topn])
        Xt = ds_all[testid]
        yt = parties_all[testid]
        At = As_all[testid]
        y_pred = self_training_model.predict(Xt)
        proba_pred = self_training_model.predict_proba(Xt)
        s_pred = np.array([np.log(i[1]*10001)- np.log(i[0]*1000) for i in proba_pred])

        acc = np.mean(y_pred == yt)
        accs.append(acc)
        
        pcc = predict_party_membership(s_pred, yt, At)
        pccs.append(pcc)
        
        
      f.write(version)

      f.write('\t' + str(accs))
      f.write('\t' + str(pccs))
      print("acc, pcc", accs, pccs)
      f.write('\n')
    else:
      accs = []
      pccs = []
      for __ in range(rounds):
        trainid, testid = sample(parties_all, theta_categ, stance_all, biased = biased, notmask = notmask, extreme = extreme)
        ys = np.copy(parties_all)
        ys[testid] = -1

        Xs = np.copy(ds_all)
        Xidx = np.arange(len(Xs))
      
        np.random.shuffle(Xidx)
        Xs = Xs[Xidx]
        ys = ys[Xidx]
        print(ys[:100])
        self_training_model.fit(Xs[:topn], ys[:topn])
        Xt = ds_all[testid]
        yt = parties_all[testid]
        At = As_all[testid]
        y_pred = self_training_model.predict(Xt)
        proba_pred = self_training_model.predict_proba(Xt)
        s_pred = np.array([np.log(i[1]*1000)- np.log(i[0]*1000) for i in proba_pred])

        acc = np.mean(y_pred == yt)
        accs.append(acc)
        
        pcc = predict_party_membership(s_pred, yt, At)
        pccs.append(pcc)
        
        
      f.write(version)

      f.write('\t' + str(accs))
      f.write('\t' + str(pccs))
      print("acc, pcc", accs, pccs)
      f.write('\n')
  f.close()

self_training_boosting_0.01_bias
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1]
test slant 0.6525330577119072 [-2.32235263  1.51016004  5.77772731 -0.90964582 -1.06624265]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1]
test slant 0.8010347504810774 [1.82613179 0.54872766 1.83041794 1.42367031 3.54231028]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1

test slant 1.163370998164613 [-2.19645658  3.78236317  3.79971014  0.04190505  2.56275709]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1]
test slant 1.099105214151821 [ 0.93631428  1.3264587   2.68394645 -0.55962863  0.0096224 ]
[ 1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1  0  0 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1  1 -1 -1 -1  1 -1 -1 -1 -1
 -1 -1 -1 -1]
test slant 0.656033794607162 [-1.53317627 -0.2256957   2.47345028  1.7136183  -0.10347767]
[-1  1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1

test slant 2.1305237784829703 [2.08650554 2.37954838 2.52009281 1.43825389 2.04143847]
[-1  1  0 -1 -1  0  0  1  1  0 -1  0  0  0  1  0  1  1 -1  1  1  1  0 -1
  0  1  0  1  0  0  0  1  1 -1 -1  0  0  1  1 -1  1  0  0 -1  0  0 -1  0
  1 -1  1 -1  0  1  1  1  1  1  0  1 -1  0  1  1  1  1  0  0  1  0  1  0
 -1  1  0  0  0  0  1  0  0  1 -1  0  1  1  0  0  0  0  0 -1  0  1  1  0
  0  0  1 -1]
test slant 2.1420990112932428 [1.92944675 2.03825392 2.44185946 2.26927785 1.88521894]
[-1  0 -1  0  1  0  0  1  1  0  0  0  0  0  1  0  1  1  0  0 -1  0  0  0
  0  0  1  0  0  0 -1 -1  0  1  0  1  0 -1  0  0  0  0 -1  0  0  1 -1  1
  0  1  0  0  1  0  1  0 -1  1  0  1  1  1  0  1  1 -1  1  1  1  0  1  1
  0  1  1 -1  1  0  1  1  1  1  0  1  1  0  0  1  1 -1  1  0  1 -1  0 -1
  0  1 -1  0]
test slant 2.037324524118592 [1.93617268 1.79065735 2.43664703 1.94507553 1.9427733 ]
[-1 -1  0  1  1  1  0  1  0  0  0  0  0  0  1  1  1  1  1  0  1  0  1  1
  0  1  0  1  0  1  1  0  0  0 -1 -1  1 -1  0  0  1  0 

test slant -1.3262581739596682 [-0.497558    1.80052258  0.39789682 -2.37647982  0.41340222]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1
 -1 -1 -1 -1]
test slant -0.75747397771786 [ 2.62987457 -2.46907256 -1.52160837 -2.3881604   0.37852183]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0  1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1  1 -1 -1 -1  0 -1  0 -1 -1 -1  0 -1 -1
 -1 -1 -1 -1]
test slant -0.9559938132650518 [-0.49531895 -3.50749154 -0.28889092 -1.59933538 -1.13500525]
self_training_boosting_0.1_unbias
[-1 -1 -1 -1  0 -1 -1  1  0 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1  1 -1 -1 -1
 -1 

test slant -0.4208508003622189 [ 0.69316757  0.99336055 -0.56573177 -1.0006869  -0.31553989]
[ 1 -1 -1  0  0 -1 -1  0  1  0  0  0 -1 -1 -1 -1  0 -1  0  1  1  1  1  0
 -1  0  0  1 -1  1 -1  1 -1  0 -1  1  0  1  0  0  1 -1  0  1  1  1  0 -1
  0  0  0 -1  1  1  0  1  0  1  0  0  1  0  0  1 -1  0  0 -1 -1 -1  1  0
 -1  1  0 -1 -1 -1  1  0  1  1  0  1 -1  1  0  0  0 -1 -1 -1 -1  1  0  0
 -1  1  0 -1]
test slant -0.3580147867714902 [-0.55148795  0.09343075 -1.01179395  0.51398214 -0.42096333]
self_training_boosting_0.8_unbias
[ 0  0  1  1  1  0 -1 -1  0  0  1  0  1  0  0  1  1  1  0 -1  0  0  0  1
  1  0  0  1  0  0  1  1  0  0  1  1  1  1  0  1  0  0  1 -1  1  1  0  0
  0  1 -1  1  0  0  1  0  0  0  0 -1  0  1  0  0  1 -1 -1  1  1  1  1  0
  0  0  0 -1 -1  1  0 -1 -1 -1 -1  0  0  0  0  0  0  1  1  1  0  0  0  0
  1  0  0 -1]
test slant -0.264854084269058 [-0.18616553 -0.56605116  0.25763439 -0.34547197  0.64718679]
[ 1  0  1  1  0  0  0 -1  0  1  0  0 -1  0 -1  1  1  1  0  0  1  0  1  1
  0

In [ ]:
# label spreading

In [ ]:
# rounds = 5
# notmasks = [0.01, 0.03, 0.05, 0.08, 0.1, 0.2, 0.4, 0.6, 0.8]

# # notmasks = [0.8]
# topn = 500000
# model_name = 'label_spreading'
# label_prop_model = LabelSpreading(kernel= 'knn', n_neighbors=10)
# # for biased in [True, False]:
# for biased in [True, False]:
#   bia = 'bias'
#   if not biased:
#     bia = 'unbias'
#   f = open( r'D:\projects\congress\results\debatepol\train\SSL/{}_{}.txt'.format(model_name, bia), 'w')
#   f.write('\taccs\tpccs\n')
#   for notmask in notmasks:
#     version = "{}_{}_{}".format(model_name, notmask, bia)
#     print(version)

#     if biased:
#       accs = []
#       pccs = []
#       for __ in range(rounds):
#         trainid, testid = sample(parties_all, theta_categ, stance_all, biased = biased, notmask = notmask, extreme = extreme)
#         ys = np.copy(parties_all)
#         ys[testid] = -1

#         Xs = np.copy(ds_all)
#         Xidx = np.arange(len(Xs))
      
#         np.random.shuffle(Xidx)
#         Xs = Xs[Xidx]
#         ys = ys[Xidx]
#         print(ys[:100])
#         label_prop_model.fit(Xs[:topn], ys[:topn])
#         Xt = ds_all[testid]
#         yt = parties_all[testid]
#         At = As_all[testid]
#         y_pred = label_prop_model.predict(Xt)
#         proba_pred = label_prop_model.predict_proba(Xt)
#         s_pred = np.array([np.log(i[1]*1000+ 0.00000001)- np.log(i[0]*1000+0.00000001) for i in proba_pred])

#         acc = np.mean(y_pred == yt)
#         accs.append(acc)
        
#         pcc = predict_party_membership(s_pred, yt, At)
#         pccs.append(pcc)
        
        
#       f.write(version)

#       f.write('\t' + str(accs))
#       f.write('\t' + str(pccs))
#       f.write('\n')
#     else:
#       accs = []
#       for __ in range(rounds):
#         trainid, testid = sample(parties_all, theta_categ, stance_all, biased = biased, notmask = notmask, extreme = extreme)
#         ys = np.copy(parties_all)
#         ys[testid] = -1

#         Xs = np.copy(ds_all)
#         Xidx = np.arange(len(Xs))
      
#         np.random.shuffle(Xidx)
#         Xs = Xs[Xidx]
#         ys = ys[Xidx]
#         print(ys[:100])
#         label_prop_model.fit(Xs[:topn], ys[:topn])
#         Xt = ds_all[testid]
#         yt = parties_all[testid]
#         At = As_all[testid]
#         y_pred = label_prop_model.predict(Xt)
#         proba_pred = label_prop_model.predict_proba(Xt)
#         s_pred = np.array([np.log(i[1]*1000+ 0.00000001)- np.log(i[0]*1000+0.00000001) for i in proba_pred])

#         acc = np.mean(y_pred == yt)
#         accs.append(acc)
        
#         pcc = predict_party_membership(s_pred, yt, At)
#         pccs.append(pcc)
        
        
#       f.write(version)

#       f.write('\t' + str(accs))
#       f.write('\t' + str(pccs))
#       f.write('\n')
#   f.close()